# INFO 4271 - Exercise 1 - Web Crawling

Issued: April 16, 2024

Due: April 22, 2024

Please submit this filled sheet via Ilias by the due date.

---

# 1. Duplicate Detection
When crawling large numbers of Web pages we are likely to encounter a considerable number of duplicate documents. To not flood our index with replicas of the same documents, we need a duplicate detection scheme.

a) Using python's built-in hash() function, process the following documents in order of appearance and flag up any exact duplicates.

- **D1** "This is just some document"
- **D2** "This is another piece of text"
- **D3** "This is another piece of text"
- **D4** "This is just some documents"
- **D5** "Totally different stuff"

In [ ]:
(hash("This is just some document") & hash("This is just some document")).bit_count

-5025021057738467412

In [49]:
len(bin(hash("This is just some document")))

66

In [ ]:
#Check a single document against an existing collection of previsouly seen documents for exact duplicates.
def check_exct(doc, docs):
    #TODO: Implement exact duplicate detection
    return hash(doc[1]) in [hash(d[1]) for d in docs] #TODO: Return True if the document is a duplicate

b) Going beyond exact duplicates, we want to also identify any near-duplicates that are very similar but not identical to previously seen content. Implement the SimHash method discussed in class and again process the five documents, this time flagging up exact and near duplicates.

In [ ]:
def create_simhash(words: list[str]):
    weights = {hash(i): words.count(i) for i in words}
    max_len = max([len(bin(w)) for w in weights.keys()])
    counts = []
    for i in range(max_len-1, -1, -1):
        count = 0
        for word, weight in weights.items():
            if i >= len(bin(word)): continue
            count += weight * (1 if bin(word)[i] == '1' else -1)
        counts.append(count)
    return bin(int(''.join(map(str, [1 if c>0 else 0 for c in counts])), 2) << 1)
#Check a single document against an existing collection of previsouly seen documents for near duplicates
def check_simhash(doc, docs):
    #TODO: Implement near duplicate detection
    doc_simhash = create_simhash(doc[1].split(" "))
    for _, compare_doc in docs:
        compare_simhash = create_simhash(compare_doc.split(" "))
        if (bin(doc_simhash) ^ bin(compare_simhash)).count("1")<10: return True
    return False #TODO: Return True if the document is a duplicate

In [82]:
crawl = [['D1', 'This is just some document'], ['D2', 'This is another piece of text'], ['D3', 'This is another piece of text'], ['D4', 'This is just some documents'], ['D5', 'Totally different stuff']]

#Process raw crawled website content
def process(crawl):
    docs = []
    for doc in crawl:
        if check_simhash(doc, docs): #Can be exchanged for check_simhash()
            print('DUPLICATE: '+doc[0])
        else:
            docs.append(doc)

process(crawl)

TypeError: unsupported operand type(s) for ^: 'str' and 'str'

In [75]:
crawl[3]

['D4', 'This is just some documents']

In [80]:
print(create_simhash(crawl[0][1].split(' ')))
print(create_simhash(crawl[3][1].split(' ')))

0b1010101010011100110001001011111011111100110110000000011010100010000
0b1010101110011100110001011011011000101100110100110000011010100010000


# 2. Focused Search Engines
Suppose you were to build a COVID-19 Web search engine for which you want to collect and eventually serve only COVID-19 information. The general web crawling process follows this scheme:

1. Create a seed set of known URLs (a.k.a the frontier)
2. Pull a URL from the frontier and visit it
3. Save the page content for our search engine (indexing)
4. Once on the page, note down all URLs linked there
5. Put all encountered URLs in the queue
6. Repeat from Step 2 until the queue is empty

In this particular setting, how should the generic step-by-step crawling process be modified/extended? Discuss all relevant considerations: